Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os

First reload the data we generated in `1_notmnist.ipynb`.

In [ ]:
root_dir = '/Users/yaluo/learning/udacity-deeplearning/'
pickle_file = os.path.join(root_dir,'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  alpha = tf.placeholder(tf.float32, shape=())
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  # Input layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  # Hidden layer.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(layer1, weights_2) + biases_2    
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
    alpha * tf.nn.l2_loss(weights_1) + alpha * tf.nn.l2_loss(weights_2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
num_steps = 3001

def evaluation(sess, w1, b1, w2, b2, dataset):
    # this essentially recreates the network for its forward pass (inference)
    # The mnist tutorial in tensorflow example has a better structure without duplicating this.
    layer1 = tf.nn.relu(tf.matmul(tf.constant(dataset), w1) + b1)
    layer2 = tf.nn.softmax(tf.matmul(layer1, w2) + b2)
    return sess.run(layer2)


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                 alpha : 0.001}
    _, lo, lg, w1, b1, w2, b2 = session.run([optimizer, loss, logits, weights_1, biases_1, weights_2, biases_2],
                                            feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, lo))
      print("Minibatch accuracy: %.1f%%" % accuracy(lg, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(evaluation(session, w1, b1, w2, b2, valid_dataset), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(evaluation(session, w1, b1, w2, b2, test_dataset), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  alpha = tf.placeholder(tf.float32, shape=())
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  # Input layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  # Hidden layer.
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(layer1, weights_2) + biases_2    
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
num_steps = 4

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                 alpha : 0.001}
    _, lo, lg, w1, b1, w2, b2 = session.run([optimizer, loss, logits, weights_1, biases_1, weights_2, biases_2],
                                            feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, lo))
      print("Minibatch accuracy: %.1f%%" % accuracy(lg, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(evaluation(session, w1, b1, w2, b2, valid_dataset), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(evaluation(session, w1, b1, w2, b2, test_dataset), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  alpha = tf.placeholder(tf.float32, shape=())
  X = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Input layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
  layer1 = tf.nn.relu(tf.matmul(X, weights_1) + biases_1)
    
  # Hidden layer.
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  layer2 = (tf.matmul(layer1, weights_2) + biases_2)
    
  # Dropout layer.
  keep_prob = tf.placeholder(tf.float32)
  drop = tf.nn.dropout(layer2, keep_prob)
  
  # Training computation.
  logits = drop
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    _, lo = session.run([optimizer, loss], feed_dict={
        X: batch_data, y: batch_labels, keep_prob: 0.5})
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, lo))
      print("  Minibatch  accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
          X: batch_data, keep_prob: 1}), batch_labels))
      print("  Validation accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
          X: valid_dataset, keep_prob: 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
      X: test_dataset, keep_prob: 1}), test_labels))


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
batch_size = 128
hidden_layer_size1 = 1024
hidden_layer_size2 = 300
hidden_layer_size3 = 50

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  alpha = tf.placeholder(tf.float32, shape=())
  X = tf.placeholder(tf.float32, shape=(None, image_size * image_size))
  y = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Input layer.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size1]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_size1]))
  layer1 = tf.nn.relu(tf.matmul(X, weights_1) + biases_1)
    
  # Hidden layer.
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size1, hidden_layer_size2]))
  biases_2 = tf.Variable(tf.zeros([hidden_layer_size2]))
  layer2 = tf.nn.relu(tf.matmul(layer1, weights_2) + biases_2)
    
  # Hidden layer 2.
  weights_3 = tf.Variable(
    tf.truncated_normal([hidden_layer_size2, num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  layer3 = (tf.matmul(layer2, weights_3) + biases_3)
    
  # Hidden layer 3.
  #weights_4 = tf.Variable(
  #  tf.truncated_normal([hidden_layer_size3, num_labels]))
  #biases_4 = tf.Variable(tf.zeros([num_labels]))
  #layer4 = tf.matmul(layer3, weights_4) + biases_4
    
  # Training computation.
  logits = layer3
  alpha = tf.placeholder(tf.float32, shape=())
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)) + alpha * (
      tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3)# + tf.nn.l2_loss(weights_4)
      )
  
  # Optimizer.
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.5, global_step, 100, 0.96)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    _, lo = session.run([optimizer, loss], feed_dict={
        X: batch_data, y: batch_labels, alpha:0.01})
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, lo))
      print("  Minibatch  accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
          X: batch_data}), batch_labels))
      print("  Validation accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
          X: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(session.run(tf.nn.softmax(logits), feed_dict={
      X: test_dataset}), test_labels))
